In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
h=0.0001

a=2.0
b=-3.0
c=10.0

d1=a*b+c
c+=h
d2=a*b+c

print('d1',d1)
print('d2',d2)
print('slope',(d2-d1)/h)

d1 4.0
d2 4.0001
slope 0.9999999999976694


In [9]:
class Value:
    def __init__(self,data) -> None:
        self.data=data

    def __repr__(self) -> str:
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        out=Value(self.data+other.data)
        return out
    
    def __mul__(self,other):
        out=Value(self.data*other.data)
        return out
    
a=Value(2.0)
b=Value(-3.0)
c=Value(10.0)
a*b


Value(data=-6.0)